In [ ]:
! pip install openai
! pip install sentence_transformers
! pip install chromadb
! pip install numpy
! pip install transformers


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached tokenizers-0.20.3-cp310-none-win_amd64.whl.metadata (6.9 kB)
Using cached tokenizers-0.20.3-cp310-none-win_amd64.whl (2.4 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.47.0 requires tokenizers<0.22,>=0.21, but you have tokenizers 0.20.3 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached tokenizers-0.21.0-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
Using cached tokenizers-0.21.0-cp39-abi3-win_amd64.whl (2.4 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chromadb 0.5.23 requires tokenizers<=0.20.3,>=0.13.2, but you have tokenizers 0.21.0 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import openai
# from sentence_transformers import SentenceTransformer
import sentence_transformers
import chromadb
from chromadb.config import Settings
from chromadb.utils import embedding_functions
import numpy as np



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\jangl\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\jangl\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\jangl\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\jangl\AppData\Local\Programs\Python\Python310\lib\site-packages\trait

AttributeError: _ARRAY_API not found

RuntimeError: Failed to import transformers.models.auto.modeling_auto because of the following error (look up to see its traceback):
Failed to import transformers.generation.utils because of the following error (look up to see its traceback):
numpy.core.multiarray failed to import

In [ ]:

# Functie voor chunking met constante grootte en overlap
def chunk_text(text, chunk_size=200, overlap=50):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = words[i:i + chunk_size]
        chunks.append(" ".join(chunk))
    return chunks

# Functie om documenten in de vectorstore te laden
def load_documents_to_vectorstore(documents, collection_name="naive_rag_collection"):
    collection = client.get_or_create_collection(
        name=collection_name,
        embedding_function=embed_text
    )
    
    for doc_id, doc_text in enumerate(documents):
        chunks = chunk_text(doc_text)
        embeddings = [embed_text(chunk) for chunk in chunks]
        
        collection.add(
            documents=chunks,
            embeddings=embeddings,
            metadatas=[{"doc_id": doc_id, "chunk_id": i} for i in range(len(chunks))]
        )

# Functie voor RAG-query

def retrieve_and_generate(query, collection_name="naive_rag_collection", top_k=5):
    collection = client.get_collection(name=collection_name)
    query_embedding = embed_text(query)

    # Retrieve de meest relevante documenten
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )

    retrieved_texts = ["\n".join(results["documents"][0])]

    # Combineer de tekst en stel een prompt samen
    context = "\n".join(retrieved_texts)
    prompt = f"Context:\n{context}\n\nVraag: {query}\n\nAntwoord:"

    # Gebruik LLM voor antwoordgeneratie
    response = openai.Completion.create(
        engine="gpt-4-mini",
        prompt=prompt,
        max_tokens=200
    )

    return response["choices"][0]["text"].strip()

# Voorbeeldgebruik
if __name__ == "__main__":
    # Documenten om te indexeren
    docs = [
        "Dit is een voorbeeldtekst over machine learning en AI.",
        "Nog een document dat ingaat op retrieval-augmented generation.",
        "Een tekst over embeddings en vectoren."
    ]

    # Laad documenten in de vectorstore
    load_documents_to_vectorstore(docs)

    # Stel een vraag
    vraag = "Wat is retrieval-augmented generation?"
    antwoord = retrieve_and_generate(vraag)
    print("Antwoord:", antwoord)
